## Preamble

Initialize some functions for better control of the code

In [1]:
import time
from memory_profiler import memory_usage

In [2]:
def timing(func, *args, **kwargs):
    start = time.time_ns()
    result = func(*args, **kwargs)
    end = time.time_ns()
    elapsed_ns = end - start
    timestamp = str(elapsed_ns)
    timestamp = timestamp.zfill((len(timestamp) // 3 + 1) * 3)
    timestamp = ' '.join(timestamp[i:i+3] for i in range(0, len(timestamp), 3))
    print(f'Time spent in {func.__name__} is {timestamp} ns')
    return result

def timing_wrap(func):
    def wrapper(*args, **kwargs):
        return timing(func, *args, **kwargs)
    return wrapper

In [3]:
def max_memory(func, *args, **kwargs):
    mem_used, result = memory_usage((func, args, kwargs), retval=True)
    print(f'Maximum memory usage in {func.__name__} is {max(mem_used)} MiB')
    return result

def max_memory_wrap(func):
    def wrapper(*args, **kwargs):
        return max_memory(func, *args, **kwargs)
    return wrapper

# 2. Introduction: theory and practice

### 2.2 Fibonacci numbers

#### Task 1

Дано целое число $1 \leq n \leq 40$, необходимо вычислить $n$-е число Фибоначчи (напомним, что $F_0=0, F_1=1, F_n=F_{n-1}+F_{n-2}$ при $n\geq2$).

**Sample Input**:
3

**Sample Output**:
2

**Time Limit**: 3 секунды
**Memory Limit**: 256 MB

##### Solution

In [4]:
def fib(n):
    fib_values = [1, 1]
    for i_n in range(2, n):
        fib_values.append(fib_values[i_n-1] + fib_values[i_n-2])
    return fib_values[n - 1]

def main():
    n = int(input())
    print(fib(n))

##### Checking

In [5]:
fib(1), fib(2), fib(3), fib(20), fib(40)

(1, 1, 2, 6765, 102334155)

#### Task 2

Дано число $1 \leq n \leq 10^7$, необходимо найти последнюю цифру $n$-го числа Фибоначчи.

Как мы помним, числа Фибоначчи растут очень быстро, поэтому при их вычислении нужно быть аккуратным с переполнением. В данной задаче, впрочем, этой проблемы можно избежать, поскольку нас интересует только последняя цифра числа Фибоначчи: если $1 \leq a, b \leq 9$ — последние цифры чисел $F_i$ и $F_{i+1}$ соответственно, то $(a+b)\ mod\ 10$ — последняя цифра числа $F_{i+2}$

**Sample Input**:
841645

**Sample Output**:
5

**Time Limit**: 3 секунды
**Memory Limit**: 256 MB

##### Solution

In [6]:
def fib_digit(n):
    n_2 = 1
    n_1 = 1
    n_0 = (n_2 + n_1) % 10
    for i_n in range(2, n):
        n_0 = (n_2 + n_1) % 10
        n_2 = n_1
        n_1 = n_0
    return n_0


def main():
    n = int(input())
    print(fib_digit(n))

##### Checking

In [7]:
fib_digit(841645)

5

#### Task 3

Даны целые числа $1 \leq n \leq 10^{18}$ и $2 \leq m \leq 10^5$, необходимо найти остаток от деления $n$-го числа Фибоначчи на $m$.

**Sample Input**:
10 2

**Sample Output**:
1

**Time Limit**: 3 секунды
**Memory Limit**: 256 MB

##### Solution

In [8]:
def fib_mod(n, m):
    v1, v2, v3 = 1, 1, 0

    for rec in bin(n)[3:]: # Matrix Exponentiation
        calc = (v2*v2) % m
        v1, v2, v3 = (v1*v1+calc) % m, ((v1+v3)*v2) % m, (calc+v3*v3) % m
        if rec == '1':
            v1, v2, v3 = (v1+v2) % m, v1, v2

    return v2

    # previous solution
    # fib_mods = [0, 1]
    # prev, curr = 0, 1
    #
    # for i_n in range(6 * m):
    #     prev, curr = curr, (prev + curr) % m
    #     fib_mods.append(curr % m)
    #     if fib_mods[-1] == 1 and fib_mods[-2] == 0:
    #         break
    # return fib_mods[n % (len(fib_mods)-2)]


def main():
    n, m = map(int, input().split())
    print(fib_mod(n, m))

##### Checking

In [9]:
timing(fib_mod, 10, 2), timing(fib_mod, 10**18, 10**5)

Time spent in fib_mod is 000 ns
Time spent in fib_mod is 000 ns


(1, 46875)

### 2.3 The greatest common divisor

#### Task 1

По данным двум числам $1 \leq a, b \leq 2 * 10^9$ найдите их наибольший общий делитель.

**Sample Input 1**:
18 35

**Sample Output 1**:
1

**Sample Input 2**:
14159572 63967072

**Sample Output 2**:
4

**Time Limit**: 3 секунды
**Memory Limit**: 256 MB

##### Solution

In [10]:
def gcd(a, b):
    if a == 0:
        return b
    if b == 0:
        return a
    if a >= b:
        return gcd(a % b, b)
    if b >= a:
        return gcd(a, b % a)


def main():
    a, b = map(int, input().split())
    print(gcd(a, b))

##### Checking

In [11]:
gcd(18, 35), timing(gcd, 14159572, 63967072)

Time spent in gcd is 000 ns


(1, 4)

### 2.4 O-notation

Completed (without code)

# 4. Greedy algorithms. Theory and Problems

### 4.1 Introduction

#### Task 1

По данным $n$ отрезкам необходимо найти множество точек минимального размера, для которого каждый из отрезков содержит хотя бы одну из точек.

В первой строке дано число $1 \leq n \leq 100$ отрезков. Каждая из последующих $n$ строк содержит по два числа $0 \leq l \leq r \leq 10^9$, задающих начало и конец отрезка. Выведите оптимальное число $m$ точек и сами $m$ точек. Если таких множеств точек несколько, выведите любое из них.

**Sample Input 1**:
3
1 3
2 5
3 6

**Sample Output 1**:
1
3

**Sample Input 2**:
4
4 7
1 3
2 5
5 6

**Sample Output 2**:
2
3 6

**Time Limit**: 1 секунда
**Memory Limit**: 256 MB

In [12]:
def task_4_1_1(segments: list) -> list:
    segments = sorted(segments, key=lambda x: x[1])
    points = [segments.pop(0)[1]]
    while len(segments) > 0:
        segment = segments.pop(0)
        if points[-1] < segment[0]:
            points.append(segment[1])
    return points

def main():
    segments_count = int(input())
    segments = []
    for _ in range(segments_count):
        segments.append(
            [int(point) for point in input().split()])
    points = task_4_1_1(segments)
    print(len(points))
    print(' '.join(map(str, points)))

In [13]:
task_4_1_1([[1, 3], [2, 5], [3, 6]]), task_4_1_1([[4, 7], [1, 3], [2, 5], [5, 6]])

([3], [3, 6])

#### Task 2

Первая строка содержит количество предметов $1 \leq n \leq 10^3$ и вместимость рюкзака $0 \leq W \leq 2 * 10^6$. Каждая из следующих $n$ строк задаёт стоимость $0 \leq c_i \leq 2 * 10^6$ и объём $0 \lt w_i \leq 2 * 10^6$ предмета ($n, W, c_i, w_i$ — целые числа). Выведите максимальную стоимость частей предметов (от каждого предмета можно отделить любую часть, стоимость и объём при этом пропорционально уменьшатся), помещающихся в данный рюкзак, с точностью не менее трёх знаков после запятой.

**Sample Input**:
3 50
60 20
100 50
120 30

**Sample Output**:
180.000

**Time Limit**: 5 секунд
**Memory Limit**: 256 MB

In [14]:
def task_4_1_2(items: list, backpack_size: int) -> list:
    items = sorted(items, key=lambda x: -x[0])
    backpack = 0
    left = backpack_size
    while left > 0 and len(items) > 0:
        cost, weight = items.pop(0)
        if weight > left:
            backpack += cost * left
        else:
            backpack += cost * weight
        left -= weight
    return backpack

def main():
    item_count, backpack_size = map(int, input().split())
    items_cost = []
    for _ in range(item_count):
        cost, weight = map(int, input().split())
        items_cost.append([cost / weight, weight])
    print(f'{task_4_1_2(items_cost, backpack_size): .3f}')

In [15]:
task_4_1_2([[3.0, 20], [2.0, 50], [4.0, 30]], 50), task_4_1_2([[3.0, 20], [2.0, 50], [4.0, 30]], 500)

(180.0, 280.0)

#### Task 3

По данному числу $1 \leq n \leq 10^9$ найдите максимальное число $k$, для которого $n$ можно представить как сумму $k$ различных натуральных слагаемых. Выведите в первой строке число $k$, во второй — $k$ слагаемых.

**Sample Input 1**:
4

**Sample Output 1**:
2
1 3

**Sample Input 2**:
6

**Sample Output 2**:
3
1 2 3

**Time Limit**: 5 секунд
**Memory Limit**: 256 MB

In [16]:
def task_4_1_3(value: int) -> list:
    if value in [1, 2]:
        return [value]
    sums = []
    left = value
    for i in range(1, value + 1):
        if left >= 2*i + 1:
            sums.append(i)
            left -= i
        else:
            sums.append(left)
            break

    return sums

def main():
    sums = task_4_1_3(int(input()))
    print(len(sums))
    print(' '.join(map(str, sums)))

In [17]:
task_4_1_3(4), task_4_1_3(14), task_4_1_3(15), task_4_1_3(9), task_4_1_3(6)

([1, 3], [1, 2, 3, 8], [1, 2, 3, 4, 5], [1, 2, 6], [1, 2, 3])

### 4.2 Huffman Codes

#### Task 1

По данной непустой строке ss длины не более $10^4$, состоящей из строчных букв латинского алфавита, постройте оптимальный беспрефиксный код. В первой строке выведите количество различных букв $k$, встречающихся в строке, и размер получившейся закодированной строки. В следующих kk строках запишите коды букв в формате "letter: code". В последней строке выведите закодированную строку.

**Sample Input 1**:
a

**Sample Output 1**:
1 1
a: 0
0

**Sample Input 2**:
abacabad

**Sample Output 2**:
4 14
a: 0
b: 10
c: 110
d: 111
01001100100111

**Time Limit**: 5 секунд
**Memory Limit**: 256 MB

In [104]:
def task_4_2_1(string: str):

    def encode():
        result = ""
        for ch in string:
            result += haff[ch]
        return result

    chars = sorted(set(string))
    if len(chars) == 1:
        haff = {chars[0]: "0"}
        return haff, encode()

    ch_sort = [(ch, string.count(ch)) for ch in set(string)]
    ch_sort.sort(key=lambda x: x[1])
    haff = {ch: "" for ch in chars}

    freq = []
    while len(ch_sort) > 1:
        l = ch_sort.pop(0)
        r = ch_sort.pop(0)
        freq.append((r[0], 0))
        freq.append((l[0], 1))
        ch_sort = [(r[0] + l[0], l[1] + r[1])] + ch_sort
        ch_sort.sort(key=lambda x: x[1])

    for str_, val in freq[::-1]:
        for ch in str_:
            haff[ch] += str(val)
    return haff, encode()

def main_4_2_1():
    d_haff, encode_str = task_4_2_1(input())
    print(len(d_haff), len(encode_str))
    for k, v in d_haff.items():
        print(k, v)
    print(encode_str)

In [105]:
task_4_2_1("abacabad"), task_4_2_1("a")

[('c', 0), ('d', 1), ('b', 0), ('cd', 1), ('a', 0), ('bcd', 1)]


(({'a': '0', 'b': '10', 'c': '110', 'd': '111'}, '01001100100111'),
 ({'a': '0'}, '0'))

In [97]:
main_4_2_1()

4 14
a 1
b 10
c 100
d 000
11011001000110


#### Task 2

Восстановите строку по её коду и беспрефиксному коду символов.

В первой строке входного файла заданы два целых числа $k$ и $l$ через пробел — количество различных букв, встречающихся в строке, и размер получившейся закодированной строки, соответственно. В следующих $k$ строках записаны коды букв в формате "letter: code". Ни один код не является префиксом другого. Буквы могут быть перечислены в любом порядке. В качестве букв могут встречаться лишь строчные буквы латинского алфавита; каждая из этих букв встречается в строке хотя бы один раз. Наконец, в последней строке записана закодированная строка. Исходная строка и коды всех букв непусты. Заданный код таков, что закодированная строка имеет минимальный возможный размер.


В первой строке выходного файла выведите строку ss. Она должна состоять из строчных букв латинского алфавита. Гарантируется, что длина правильного ответа не превосходит $10^4$ символов.

**Sample Input 1**:
1 1
a: 0
0

**Sample Output 1**:
a

**Sample Input 2**:
4 14
a: 0
b: 10
c: 110
d: 111
01001100100111

**Sample Output 2**:
abacabad

**Time Limit**: 5 секунд
**Memory Limit**: 256 MB

In [117]:
def task_4_2_2(haff: dict, string: str):
    result = ""
    key = ""
    for ch in string:
        key += ch
        if key in haff:
            result += haff[key]
            key = ""
    return result

def main_4_2_2():
    len_haff, len_str = tuple(map(int, input().split()))
    haff = {}
    for _ in range(len_haff):
        ch, code = input().split()
        haff[code] = ch[:-1]
    string = input()
    print(task_4_2_2(haff, string))

In [118]:
task_4_2_2({'0': 'a', '10': 'b', '110': 'c', '111': 'd'}, "01001100100111")

'abacabad'